# Web Scrapping para boletines de campaña de AMLO

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
# from tqdm.notebook import tqdm
from tqdm import tqdm
from uuid import uuid4
import pandas as pd
import datetime as dt

## Definición de constantes

In [4]:
# FECHA_INICIO = dt.date(2018, 3, 30)
FECHA_INICIO = dt.date(2018, 4, 1)
FECHA_FIN = dt.date(2018, 6, 27)
UN_DIA = dt.timedelta(days=1)
DIAS_DIF = (FECHA_FIN - FECHA_INICIO).days + 1
SITIO_BASE = 'https://lopezobrador.org.mx/'
RUTA_CHROMEDRIVER = "C:\\Chromedriver\\chromedriver.exe"
articulos = []

def obtener_ruta(fecha):
    return SITIO_BASE + fecha.strftime("%Y/%m/%d")

# print('Inicio de campaña: ', FECHA_INICIO)
# print('Fin de campaña: ', FECHA_FIN)

## Driver chrome


In [3]:
driver = webdriver.Chrome(RUTA_CHROMEDRIVER)

## Scrapping
Lógica principal para obtener información

In [6]:
rutas = [obtener_ruta(FECHA_INICIO + UN_DIA*i) for i in range(DIAS_DIF)]
for ruta in tqdm(rutas, desc='Progreso por fecha'):
#     ruta_actual = obtener_ruta(fecha_actual)
    driver.get(ruta)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    # Buscamos todos los artículos con clase boletin
    for soup_articulo in soup.findAll('article', attrs={'class':'category-boletines'}):
        articulo = {
            'uuid': str(uuid4()),
            'fuente' : '',
            'encabezado' : '',
            'sumarios' : [],
            'contenido' : [],
        }
        
        # Encabezado del boletin
        soup_encabezado = soup_articulo.find('h2', attrs={'class':'entry-title'})
        articulo['encabezado'] = soup_encabezado.a.string
        articulo['fuente'] = soup_encabezado.a.attrs['href']
        
        # Sumarios del boletin
        soup_sumarios = soup_articulo.find('ul')
        if soup_sumarios:
            for soup_sumario in soup_sumarios.findAll('li'):
                sumario = soup_sumario.string if soup_sumario.string else ""
                if( len(sumario) < 3 ):
                    sumario = " ".join([child.string for child in soup_sumario.children if child.string])
                sumario = sumario.strip()
                if( len(sumario) >= 3):
                    articulo['sumarios'].append(sumario)
                
        # Contenido del boletin
        soup_content = soup_articulo.find('div', attrs={'class':'entry-content'})
        for soup_p in soup_content.findAll('p'):
            contenido = soup_p.string if soup_p.string else ""
            if( len(contenido) < 3 ):
                contenido = " ".join([child.string for child in soup_p.children if child.string])
            contenido = contenido.strip()
            if( len(contenido) >= 3 and contenido != 'ªªªªªªªªªªª' ):
                articulo['contenido'].append(contenido)
        
        # Agregamos el nuevo artículo
        articulos.append(articulo)
#     fecha_actual += UN_DIA

# Hacemos un objeto    
boletines = {
    'boletines' : articulos
}

Progreso por fecha: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [02:28<00:00,  1.43s/it]


In [7]:
df_boletines = pd.DataFrame(boletines)

In [8]:
with open('Data/boletines-AMLO.json', 'w', encoding='utf-8') as file:
    df_boletines.to_json(file, force_ascii=False)